In [ ]:
!pip install requests bs4 re

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
elements = []

# Access the web
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_PC_games_(Numerical)').text
soup = BeautifulSoup(website_url,'lxml')

table = soup.find('table', class_='wikitable sortable')

for row in table.find_all('tr'):
    try:
        game = row.find_all('i')
        game_name = game[0].find(string=True)
        game_link = game[0].find(href=True)['href']
    except:
        pass
    else:
        cells = row.find_all('td')
        atts = []
        for i in range(6):
            att = cells[i].find(string=True)
            att = att.replace('\n', ' ')
            att = att.replace('\t', ' ')
            atts.append(att)

        elements.append([game_link] + atts)
cols = ['Link', 'Title', 'Developer', 'Publisher','Genre','Operating System','Release Date']
df_games = pd.DataFrame(elements, columns=cols)
df_games = df_games.astype(str)

# Clean links and remove Genre column
# df_games = df_games.applymap(lambda x: x.replace('\n', ''))
# df_games = df_games.applymap(lambda x: x.replace(':', ''))
# df_games.drop(columns=['Genre'], inplace=True)

wiki_url = 'https://en.wikipedia.org'
plots = []
gameplays = []

for idx, row in df_games.iterrows():
    url = wiki_url + row['Link']
    website_url = requests.get(url).text
    soup = BeautifulSoup(website_url,'lxml')
    text = ''
    gameplays_text = ''

    for section in soup.find_all('h2'):
        if section.text.startswith('Plot'):
            text += section.text + '\n\n'
            for element in section.next_siblings:
                if element.name and element.name.startswith('h2'):
                    break
                elif element.name == 'p':
                    text += element.text + '\n'
        else:
            pass

    if not text:
        plots.append(None)
    else:
        plots.append(text)

    for section in soup.find_all('h2'):
      if section.text.startswith('Game'):
          gameplays_text += section.text + '\n\n'
          for element in section.next_siblings:
              if element.name and element.name.startswith('h2'):
                  break
              elif element.name == 'p':
                  gameplays_text += element.text + '\n'
      else:
          pass

    if not gameplays_text:
      gameplays.append(None)
    else:
      gameplays.append(gameplays_text)


plots_clean = []
gameplays_clean = []

for text in plots:
    if text is not None:
        text = re.sub(r'\[.*?\]+', '', text)
        text = text.replace('\n', ' ')
        text = text.replace('Gameplay ', '')
        text = text.replace('Game-play ', '')
        text = text.replace('Plot ', '')
        plots_clean.append(text)
    else:
        plots_clean.append(None)

for text in gameplays:
    if text is not None:
        text = re.sub(r'\[.*?\]+', '', text)
        text = text.replace('\n', ' ')
        text = text.replace('Gameplay ', '')
        text = text.replace('Game-play ', '')
        text = text.replace('Plot ', '')
        gameplays_clean.append(text)
    else:
        gameplays_clean.append(None)

df_games['Plots'] = plots_clean
df_games['Gameplays'] = gameplays_clean
# Xóa các hàng có cột 'Plots' hoặc cột 'Gameplays' là None
df_games.dropna(subset=['Plots'], inplace=True)
df_games.dropna(subset=['Gameplays'], inplace=True)
df_games = df_games[df_games['Plots'] != '']
df_games = df_games[df_games['Gameplays'] != '']

# Reset lại chỉ mục sau khi xóa
df_games.reset_index(drop=True, inplace=True)

idx_todrop = df_games[df_games.Title=='Untitled '].index.tolist()
df_games.drop(index=idx_todrop, inplace=True)

import os.path

# Đường dẫn đến tập tin CSV hiện có
file_path = 'datasets/Games_dataset.csv'

# Kiểm tra xem tập tin đã tồn tại chưa
if os.path.exists(file_path):
    # Đọc dữ liệu từ tập tin CSV hiện có thành DataFrame
    existing_df = pd.read_csv(file_path)

    # Nối DataFrame hiện tại với DataFrame đã đọc từ tập tin CSV
    updated_df = pd.concat([existing_df, df_games], ignore_index=True)
else:
    # Nếu tập tin không tồn tại, sử dụng DataFrame hiện tại
    updated_df = df_games

# Lưu DataFrame đã cập nhật vào tập tin CSV
updated_df.to_csv(file_path, index=False)

In [ ]:
df_games

,Link,Title,Developer,Publisher,Genre,Operating System,Release Date,Plots,Gameplays
0,/wiki/A.R.E.S.:_Extinction_Agenda,A.R.E.S.: Extinction Agenda,Extend Studio,"Origo Games,",Action,Microsoft Windows,"Dec 14, 2010",The game takes place in a future setting wher...,Players control Ares' movement with WASD keys...
1,/wiki/A_Hat_in_Time,A Hat in Time,Gears for Breakfast,Gears for Breakfast,Platform,macOS,"Oct 5, 2017","A Hat in Time follows Hat Kid, a little alien...",A Hat in Time is a platform game set in an op...
2,/wiki/Accel_World_vs._Sword_Art_Online:_Millen...,Accel World vs. Sword Art Online: Millennium T...,Artdink,Bandai Namco Entertainment,Action role-playing,Microsoft Windows,"Sep 12, 2017",The story begins with Asuna and Kirito from S...,Accel World vs. Sword Art Online is a team-ba...
3,/wiki/Ace_Combat:_Assault_Horizon,Ace Combat: Assault Horizon,Project Aces,Namco Bandai Games,Combat flight simulator,Microsoft Windows,"Jan 24, 2013",Unlike other games in the Ace Combat series (...,The major new gameplay feature is a system ca...
4,/wiki/Achron,Achron,"Hazardous Software, Inc.","Hazardous Software, Inc.",Real-time strategy,Microsoft Windows,"Aug 29, 2011","Hundreds of years in the future, humans have ...","The main aspect in Achron is the use of ""chro..."
5,/wiki/Act_of_War:_Direct_Action,Act of War: Direct Action,Eugen Systems,"Atari, Inc.",Real-time Strategy (RTS),Microsoft Windows,"Mar 15, 2005",Act of War is set in what is only referred to...,The game includes a combination of pre-render...
6,/wiki/Adventure_at_the_Chateau_d%27Or,Adventure at the Chateau d'Or,Karma Labs,Karma Labs,Adventure,Microsoft Windows,"Apr 2, 2001",The player has been quested by a beautiful pr...,is similar to Myst involving the player navig...
7,/wiki/Adventures_of_Pip,Adventures of Pip,Tic Toc Games,Tic Toc Games,Action,Microsoft Windows,"Jun 4, 2015",An evil witch has come and turned the King an...,"In Adventures of Pip, the player plays as Pip..."
8,/wiki/Age_of_Empires_III,Age of Empires III,Ensemble Studios,Microsoft Game Studios,Real-time Strategy (RTS),Microsoft Windows,"Oct 18, 2005",The campaign follows the fictional Black fami...,Players begin with a constructed town center ...
9,/wiki/Age_of_Empires_III:_The_WarChiefs,Age of Empires III: The WarChiefs,Ensemble Studios,Microsoft Game Studios,Real-time Strategy (RTS),Microsoft Windows,"Oct 7, 2006","The campaign, which is 15 missions long, incl...",changes There are no major changes to the mec...


In [ ]:
df_games.dropna().to_csv('datasets/Games_dataset.csv', index=False)